# Human in the Loop

There are many occasions where you would like to put a human in the loop to keep tabs on what an agent is doing
.


# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

# Global Imports

In [3]:
import operator
from typing import Annotated, TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

In [4]:
memory = SqliteSaver.from_conn_string(":memory:")

> **We're gonna perform a small modification**.

In the previous examples, we annotated the messages list with the `operator.add`. That added messages to the existing messages array. 

However, for this human-in-the-loop interactions, **we may want to actually replace these existing messages**.

In order to do that, we're going to write a custom `reduce_messages` function, that basically looks for messages with the same id, and appends them otherwise.

In [5]:
from uuid import uuid4

from langchain_core.messages import AIMessage

In [7]:
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # Assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # Merge the new messages (from the right) to the existing ones (from the left)
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # Replace any existing message (from the left)
            # By one with the same id (from the right)
            if existing.id == message.id:
                merged[i] = message
                break
        # This `else` clause suggests no `break` was encountered, it's not mandatory
        else:
            # Append any new message to the end
            merged.append(message)
    return merged

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]